参考链接：
- [5-4，模型层layers](https://www.kesci.com/home/project/5ea7d3b2564b12002c09dc57)

## 自定义模型层

如果自定义模型层没有需要被训练的参数，一般推荐使用Lamda层实现。

如果自定义模型层有需要被训练的参数，则可以通过对Layer基类子类化实现。

Lamda层由于没有需要被训练的参数，只需要定义正向传播逻辑即可，使用比Layer基类子类化更加简单。

Lamda层的正向逻辑可以使用Python的lambda函数来表达，也可以用def关键字定义函数来表达。

In [3]:
import tensorflow as tf
from tensorflow.keras import layers,models,regularizers

tf.__version__

'2.2.0'

In [4]:
mypower = layers.Lambda(lambda x:tf.math.pow(x,2))
mypower(tf.range(5))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 0,  1,  4,  9, 16])>

Layer的子类化一般需要重新实现初始化方法，Build方法和Call方法。下面是一个简化的线性层的范例，类似Dense.

In [6]:
class Linear(layers.Layer):
    def __init__(self,units=32,**kwargs):
        super(Linear,self).__init__(**kwargs)
        self.units=units
    def build(self,input_shape):
        self.w=self.add_weight(shape=(input_shape[-1],self.units),
                               initializer='random_normal',
                               trainable=True)
        self.b=self.add_weight(shape=(self.units,),
                              initializer='random_normal',
                              trainable=True)
        super(Linear,self).build(input_shape)
    def call(self,inputs):
        """
        call方法一般定义定义正向1传播运算逻辑，__call__方法调用此方法
        """
        return tf.matmul(inputs,self.w)+self.b
    
    def get_config(self):
        config=super(Linear,self).get_config()
        config.update({'units':self.units})
        return config

In [7]:
linear = Linear(units = 8)
print(linear.built)
#指定input_shape，显式调用build方法，第0维代表样本数量，用None填充
linear.build(input_shape = (None,16)) 
print(linear.built)

False
True


In [8]:
linear = Linear(units = 8)
print(linear.built)
linear.build(input_shape = (None,16)) 
print(linear.compute_output_shape(input_shape = (None,16)))

False
(None, 8)


In [9]:
linear = Linear(units = 16)
print(linear.built)
#如果built = False，调用__call__时会先调用build方法, 再调用call方法。
linear(tf.random.uniform((100,64))) 
print(linear.built)
config = linear.get_config()
print(config)

False
True
{'name': 'linear_2', 'trainable': True, 'dtype': 'float32', 'units': 16}


In [10]:
tf.keras.backend.clear_session()

model = models.Sequential()
#注意该处的input_shape会被模型加工，无需使用None代表样本数量维
model.add(Linear(units = 16,input_shape = (64,)))  
print("model.input_shape: ",model.input_shape)
print("model.output_shape: ",model.output_shape)
model.summary()

model.input_shape:  (None, 64)
model.output_shape:  (None, 16)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear (Linear)              (None, 16)                1040      
Total params: 1,040
Trainable params: 1,040
Non-trainable params: 0
_________________________________________________________________
